In [ ]:
!pip install pennylane torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 20.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl

In [ ]:
def split_text_file_by_last_digit(filename):
  """
  Splits a text file into separate files based on the last digit value in each line.

  Args:
    filename: The name of the text file to split.
  """

  # Create a dictionary to store the separate files
  files = {}

  # Open the text file for reading
  with open(filename, 'r') as f:
    # Iterate through each line in the file
    for line in f:
      # Extract the last digit from the line
      last_digit = line[-2]

      # Check if a file already exists for the last digit
      if last_digit not in files:
        # Create a new file for the last digit
        files[last_digit] = open(filename + last_digit, 'w')

      # Write the line to the corresponding file
      files[last_digit].write(line[:-2])

  # Close all the files
  for file in files.values():
    file.close()

# Split the text file
split_text_file_by_last_digit('/content/AI_2qubits_training_data.txt')

In [ ]:
# Library imports
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pennylane as qml

# Pytorch imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

# Set the random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset

class BitstringsDataset(Dataset):
    """Pytorch dataloader for a list of bitstrings"""

    def __init__(self, txt_file, transform=None):
        """
        Args:
            txt_file (string): Path to the text file with bitstrings.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.txt_file = txt_file
        self.transform = transform
        self.df = self.bitstrings_to_dataframe()

    def bitstrings_to_dataframe(self):
      # Step 2: Strip the newline characters from each string
        lines = [line.strip() for line in self.txt_file]

        # Step 3: Create a pandas DataFrame from the list
        df = pd.DataFrame(lines)
        return df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # string = self.df.iloc[idx, :-1] / 16
        # string = np.array(string)
        # string = string.astype(np.float32).reshape(8, 8)

        if self.transform:
            string = self.transform(self.txt_file)

        # Return image and label
        return string

In [ ]:
image_size = 8  # Height / width of the square images
batch_size = 1

transform = transforms.Compose([transforms.ToTensor()])
dataset = BitstringsDataset(txt_file="/content/1-AI_2qubits_training_data.txt", transform=transform)
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, shuffle=True, drop_last=True
)

In [ ]:
print(dataset[0])

TypeError: pic should be PIL Image or ndarray. Got <class 'str'>

In [ ]:
# image_size = 8  # Height / width of the square images
# batch_size = 1



# transform = transforms.Compose([transforms.ToTensor()])
# dataset = DigitsDataset(csv_file="/content/optdigits.tra", transform=transform)
# dataloader = torch.utils.data.DataLoader(
#     dataset, batch_size=batch_size, shuffle=True, drop_last=True
# )

In [ ]:
plt.figure(figsize=(8,2))
for i in range(8):
    print(dataset[i])
    # image = dataset[i][0].reshape(image_size,image_size)
    # plt.subplot(1,8,i+1)
    # plt.axis('off')
    # plt.imshow(image.numpy(), cmap='gray')

# plt.show()

UnboundLocalError: local variable 'string' referenced before assignment

<Figure size 800x200 with 0 Axes>

In [ ]:
class Discriminator(nn.Module):
    """Fully connected classical discriminator"""

    def __init__(self):
        super().__init__()

        self.model = nn.Sequential(
            # Inputs to first hidden layer (num_input_features -> 64)
            nn.Linear(image_size * image_size, 64),
            nn.ReLU(),
            # First hidden layer (64 -> 16)
            nn.Linear(64, 16),
            nn.ReLU(),
            # Second hidden layer (16 -> output)
            nn.Linear(16, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Quantum variables
n_qubits = 5  # Total number of qubits / N
n_a_qubits = 1  # Number of ancillary qubits / N_A
q_depth = 6  # Depth of the parameterised quantum circuit / D
n_generators = 4  # Number of subgenerators for the patch method / N_G

In [ ]:
# Quantum simulator
dev = qml.device("lightning.qubit", wires=n_qubits)
# Enable CUDA device if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
@qml.qnode(dev, diff_method="parameter-shift")
def quantum_circuit(noise, weights):

    weights = weights.reshape(q_depth, n_qubits)

    # Initialise latent vectors
    for i in range(n_qubits):
        qml.RY(noise[i], wires=i)

    # Repeated layer
    for i in range(q_depth):
        # Parameterised layer
        for y in range(n_qubits):
            qml.RY(weights[i][y], wires=y)

        # Control Z gates
        for y in range(n_qubits - 1):
            qml.CZ(wires=[y, y + 1])

    return qml.probs(wires=list(range(n_qubits)))


# For further info on how the non-linear transform is implemented in Pennylane
# https://discuss.pennylane.ai/t/ancillary-subsystem-measurement-then-trace-out/1532
def partial_measure(noise, weights):
    # Non-linear Transform
    probs = quantum_circuit(noise, weights)
    probsgiven0 = probs[: (2 ** (n_qubits - n_a_qubits))]
    probsgiven0 /= torch.sum(probs)

    # Post-Processing
    probsgiven = probsgiven0 / torch.max(probsgiven0)
    return probsgiven

In [ ]:
class PatchQuantumGenerator(nn.Module):
    """Quantum generator class for the patch method"""

    def __init__(self, n_generators, q_delta=1):
        """
        Args:
            n_generators (int): Number of sub-generators to be used in the patch method.
            q_delta (float, optional): Spread of the random distribution for parameter initialisation.
        """

        super().__init__()

        self.q_params = nn.ParameterList(
            [
                nn.Parameter(q_delta * torch.rand(q_depth * n_qubits), requires_grad=True)
                for _ in range(n_generators)
            ]
        )
        self.n_generators = n_generators

    def forward(self, x):
        # Size of each sub-generator output
        patch_size = 2 ** (n_qubits - n_a_qubits)

        # Create a Tensor to 'catch' a batch of images from the for loop. x.size(0) is the batch size.
        images = torch.Tensor(x.size(0), 0).to(device)

        # Iterate over all sub-generators
        for params in self.q_params:

            # Create a Tensor to 'catch' a batch of the patches from a single sub-generator
            patches = torch.Tensor(0, patch_size).to(device)
            for elem in x:
                q_out = partial_measure(elem, params).float().unsqueeze(0)
                patches = torch.cat((patches, q_out))

            # Each batch of patches is concatenated with each other to create a batch of images
            images = torch.cat((images, patches), 1)

        return images

In [ ]:
lrG = 0.3  # Learning rate for the generator
lrD = 0.01  # Learning rate for the discriminator
num_iter = 500  # Number of training iterations

In [ ]:
discriminator = Discriminator().to(device)
generator = PatchQuantumGenerator(n_generators).to(device)

# Binary cross entropy
criterion = nn.BCELoss()

# Optimisers
optD = optim.SGD(discriminator.parameters(), lr=lrD)
optG = optim.SGD(generator.parameters(), lr=lrG)

real_labels = torch.full((batch_size,), 1.0, dtype=torch.float, device=device)
fake_labels = torch.full((batch_size,), 0.0, dtype=torch.float, device=device)

# Fixed noise allows us to visually track the generated images throughout training
fixed_noise = torch.rand(8, n_qubits, device=device) * math.pi / 2

# Iteration counter
counter = 0

# Collect images for plotting later
results = []

while True:
    for i, (data, _) in enumerate(dataloader):

        # Data for training the discriminator
        data = data.reshape(-1, image_size * image_size)
        real_data = data.to(device)

        # Noise follwing a uniform distribution in range [0,pi/2)
        noise = torch.rand(batch_size, n_qubits, device=device) * math.pi / 2
        fake_data = generator(noise)

        # Training the discriminator
        discriminator.zero_grad()
        outD_real = discriminator(real_data).view(-1)
        outD_fake = discriminator(fake_data.detach()).view(-1)

        errD_real = criterion(outD_real, real_labels)
        errD_fake = criterion(outD_fake, fake_labels)
        # Propagate gradients
        errD_real.backward()
        errD_fake.backward()

        errD = errD_real + errD_fake
        optD.step()

        # Training the generator
        generator.zero_grad()
        outD_fake = discriminator(fake_data).view(-1)
        errG = criterion(outD_fake, real_labels)
        errG.backward()
        optG.step()

        counter += 1

        # Show loss values
        if counter % 10 == 0:
            print(f'Iteration: {counter}, Discriminator Loss: {errD:0.3f}, Generator Loss: {errG:0.3f}')
            test_images = generator(fixed_noise).view(8,1,image_size,image_size).cpu().detach()

            # Save images every 50 iterations
            if counter % 50 == 0:
                results.append(test_images)

        if counter == num_iter:
            break
    if counter == num_iter:
        break

Iteration: 10, Discriminator Loss: 1.356, Generator Loss: 0.597
Iteration: 20, Discriminator Loss: 1.325, Generator Loss: 0.621
Iteration: 30, Discriminator Loss: 1.333, Generator Loss: 0.615
Iteration: 40, Discriminator Loss: 1.304, Generator Loss: 0.633
Iteration: 50, Discriminator Loss: 1.275, Generator Loss: 0.648
Iteration: 60, Discriminator Loss: 1.222, Generator Loss: 0.670
Iteration: 70, Discriminator Loss: 1.267, Generator Loss: 0.627
Iteration: 80, Discriminator Loss: 1.253, Generator Loss: 0.647
Iteration: 90, Discriminator Loss: 1.256, Generator Loss: 0.624
Iteration: 100, Discriminator Loss: 1.249, Generator Loss: 0.627
Iteration: 110, Discriminator Loss: 1.177, Generator Loss: 0.660
Iteration: 120, Discriminator Loss: 1.175, Generator Loss: 0.633
Iteration: 130, Discriminator Loss: 1.210, Generator Loss: 0.664
Iteration: 140, Discriminator Loss: 1.231, Generator Loss: 0.605
Iteration: 150, Discriminator Loss: 1.252, Generator Loss: 0.606
Iteration: 160, Discriminator Loss

In [ ]:
fig = plt.figure(figsize=(10, 5))
outer = gridspec.GridSpec(5, 2, wspace=0.1)

for i, images in enumerate(results):
    inner = gridspec.GridSpecFromSubplotSpec(1, images.size(0),
                    subplot_spec=outer[i])

    images = torch.squeeze(images, dim=1)
    for j, im in enumerate(images):

        ax = plt.Subplot(fig, inner[j])
        ax.imshow(im.numpy(), cmap="gray")
        ax.set_xticks([])
        ax.set_yticks([])
        if j==0:
            ax.set_title(f'Iteration {50+i*50}', loc='left')
        fig.add_subplot(ax)

plt.show()

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

def load_data(file_path):
    with open(file_path, 'r') as file:
        # Read lines from the file and split each line by space
        binary_data = [line.split(' ') for line in file.read().splitlines()]
        binary_data = list(filter(None, binary_data[0]))

    data = np.array(binary_data)
    print(data)
    return data

# Define the generator
def build_generator():
    model = Sequential([
        Dense(units=128, input_dim=100, activation='relu'),
        Dense(units=100, activation='sigmoid')  # Output is a 100-digit binary number
    ])
    return model

# Define the discriminator
def build_discriminator():
    model = Sequential([
        Dense(units=128, input_dim=100, activation='relu'),
        Dense(units=1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
    return model

# Define the GAN
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential([generator, discriminator])
    model.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
    return model

# Training the GAN
def train_gan(generator, discriminator, gan, data, epochs, batch_size):
    for epoch in range(epochs):
        # Random noise for generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        # Generate fake data
        gen_data = generator.predict(noise)
        print(data.shape)
        # Get a random batch of real data
        idx = np.random.randint(0, data.shape[0], batch_size)
        real_data = data[idx]

        # Train discriminator
        d_loss_real = discriminator.train_on_batch(real_data, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_data, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        # Print progress
        print(f"Epoch: {epoch+1}/{epochs} | D Loss: {d_loss[0]} | D Acc: {100*d_loss[1]} | G Loss: {g_loss}")

# Load data
data = load_data('/content/1-AI_2qubits_training_data.txt')

# Initialize models
generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

# Set training parameters
epochs = 10000
batch_size = 32

# Train GAN
train_gan(generator, discriminator, gan, data, epochs, batch_size)

['0100111111110000000000101110100011011011110000110101001001111101000110011111011100111010001111110010'
 '0110011011110001111110001000001110011110011100110100001100011101100100001101111111010000000110001101'
 '1110100101110111111101101100110101111101010001011011110011010001001101100110101011110110000110000011'
 ...
 '1100101101110111011000000101001100110111010010111100010001110010110000001110000101001111101001001011'
 '0101111100000011010110010111101111001110010111111100101001001110110000100011100011011011000010000000'
 '1101101001001011111001000101011110010111011111111101001110011011001011101000100111100000010101000100']
1/1 [==============================] - 0s 101ms/step
(2000,)


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 253, in assert_input_compatibility
        raise ValueError(

    ValueError: Ex

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Load the dataset
def load_data(file_path):
    with open(file_path, 'r') as file:
        # Read lines from the file and split each line by space
        binary_data = [line.split(' ') for line in file.read().splitlines()]
        binary_data = list(filter(None, binary_data[0]))

    data = np.array(binary_data, dtype=float)
    data = torch.tensor(data, dtype=torch.float32)
    print(data)
    return data

# Define the generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 128),
            nn.ReLU(),
            nn.Linear(128, 100),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Define the discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Training the GAN
def train_gan(generator, discriminator, data, epochs, batch_size):
    # Optimizers
    d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    g_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))

    # Loss function
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        # Train Discriminator
        discriminator.zero_grad()

        # Real data
        real_data = data[torch.randint(0, data.size(0), (batch_size,))]
        real_labels = torch.ones(batch_size, 1)
        real_output = discriminator(real_data)
        d_loss_real = criterion(real_output, real_labels)

        # Fake data
        noise = torch.randn(batch_size, 100)
        fake_data = generator(noise)
        fake_labels = torch.zeros(batch_size, 1)
        fake_output = discriminator(fake_data.detach())
        d_loss_fake = criterion(fake_output, fake_labels)

        # Discriminator loss
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        d_optimizer.step()

        # Train Generator
        generator.zero_grad()
        output = discriminator(fake_data)
        g_loss = criterion(output, real_labels)
        g_loss.backward()
        g_optimizer.step()

        if epoch % 100 == 0:
            print(f"Epoch: {epoch}/{epochs} | D Loss: {d_loss.item()} | G Loss: {g_loss.item()}")

# Load data
data = load_data('1-AI_2qubits_training_data.txt')

# Initialize models
generator = Generator()
discriminator = Discriminator()

# Set training parameters
epochs = 10000
batch_size = 100

# Train GAN
train_gan(generator, discriminator, data, epochs, batch_size)

tensor([inf, inf, inf,  ..., inf, inf, inf])


ValueError: Using a target size (torch.Size([100, 1])) that is different to the input size (torch.Size([1])) is deprecated. Please ensure they have the same size.

In [ ]:
print(len)

2001


In [ ]:
len("0110011011110001111110001000001110011110011100110100001100011101100100001101111111010000000110001101")

100